# Tutorial 4 - Setting parameter values

In [Tutorial 1](./Tutorial%201%20-%20How%20to%20run%20a%20model.ipynb) and [Tutorial 2](./Tutorial%202%20-%20Compare%20models.ipynb), we saw how to run a PyBaMM model with all the default settings. However, PyBaMM also allows you to tweak these settings for your application. In this tutorial, we will see how to change the parameters in PyBaMM.

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import os
os.chdir(pybamm.__path__[0]+'/..')

Note: you may need to restart the kernel to use updated packages.


## Change the whole parameter set

PyBaMM has a number of in-built parameter sets (check the list [here](https://pybamm.readthedocs.io/en/latest/source/parameters/parameter_sets.html)), which can be selected doing

In [2]:
chemistry = pybamm.parameter_sets.Chen2020

This variable is a dictionary with the corresponding parameter subsets for each component.

In [3]:
chemistry

{'chemistry': 'lithium-ion',
 'cell': 'LGM50_Chen2020',
 'anode': 'graphite_Chen2020',
 'separator': 'separator_Chen2020',
 'cathode': 'nmc_Chen2020',
 'electrolyte': 'lipf6_Nyman2008',
 'experiment': '1C_discharge_from_full_Chen2020',
 'sei': 'example',
 'citation': 'Chen2020'}

More details on each subset can be found [here](https://github.com/pybamm-team/PyBaMM/tree/master/pybamm/input/parameters).

Now we can pass `'chemistry'` into `ParameterValues` to create the dictionary of parameter values

In [4]:
parameter_values = pybamm.ParameterValues(chemistry=chemistry)

We can see all the parameters stored in the dictionary

In [5]:
parameter_values

{'1 + dlnf/dlnc': 1.0,
 'Ambient temperature [K]': 298.15,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell capacity [A.h]': 5.0,
 'Cell cooling surface area [m2]': 0.0053100000000000005,
 'Cell volume [m3]': 2.42e-05,
 'Current function [A]': 5.0,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008 at 0x7f4fbd88a620>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008 at 0x7f500fac02f0>,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 29866.0,
 'Initial concentration in positive electrode [mol.m-3]': 17038.0,
 'Initial inner SEI thickness [m]': 2.5e-09,
 'Initial outer SEI thickness [m]': 2.5e-09,
 'Initial temperature [K]': 298.15,
 'Inner SEI electron con

or we can search for a particular parameter

In [6]:
parameter_values.search("electrolyte")

EC initial concentration in electrolyte [mol.m-3]	4541.0
Electrolyte conductivity [S.m-1]	<function electrolyte_conductivity_Nyman2008 at 0x7f4fbd88a620>
Electrolyte diffusivity [m2.s-1]	<function electrolyte_diffusivity_Nyman2008 at 0x7f500fac02f0>
Initial concentration in electrolyte [mol.m-3]	1000.0
Negative electrode Bruggeman coefficient (electrolyte)	1.5
Positive electrode Bruggeman coefficient (electrolyte)	1.5
Separator Bruggeman coefficient (electrolyte)	1.5
Typical electrolyte concentration [mol.m-3]	1000.0


To run a simulation with this parameter set, we can proceed as usual but passing the parameters as a keyword argument

In [7]:
model = pybamm.lithium_ion.DFN()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=3527.2727272727275, step=35.27272727272727),…

## Change individual parameters

We often want to quickly change a small number of parameter values to investigate how the behaviour or the battery changes. In such cases, we can change parameter values without having to leave the notebook or script you are working in. 

We start initialising the model and the parameter values

In [8]:
model = pybamm.lithium_ion.DFN()
parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Chen2020)

In this example we will change the current to 10 A

In [9]:
parameter_values["Current function [A]"] = 10

Now we just need to run the simulation with the new parameter values

In [10]:
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve([0, 3600])
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1672.727272727273, step=16.72727272727273), …

Note that we still passed the interval `[0, 3600]` to `sim.solve()`, but the simulation terminated early as the lower voltage cut-off was reached.

### Drive cycle

You can implement drive cycles importing the dataset and creating an interpolant to pass as the current function.

In [11]:
import pandas as pd    # needed to read the csv data file

# Import drive cycle from file
drive_cycle = pd.read_csv("pybamm/input/drive_cycles/US06.csv", comment="#", header=None).to_numpy()

# Create interpolant
timescale = parameter_values.evaluate(model.timescale)
current_interpolant = pybamm.Interpolant(drive_cycle, timescale * pybamm.t)

# Set drive cycle
parameter_values["Current function [A]"] = current_interpolant

Note that your drive cycle data can be stored anywhere, you just need to pass the path of the file. Then, again, the model can be solved as usual but notice that now, if `t_eval` is not specified, the solver will take the time points from the data set.

In [12]:
model = pybamm.lithium_ion.SPMe()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
sim.solve()
sim.plot(["Current [A]", "Terminal voltage [V]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=600.0, step=6.0), Output()), _dom_classes=('…

### Custom current function

Alternatively, we can define the current to be an arbitrary function of time

In [13]:
import numpy as np

def my_current(t):
    return pybamm.sin(2 * np.pi * t / 60)

parameter_values["Current function [A]"] = my_current

and we can now solve the model again. In this case, we can pass `t_eval` to the solver to make sure we have enough time points to resolve the function in our output.

In [14]:
model = pybamm.lithium_ion.SPMe()
sim = pybamm.Simulation(model, parameter_values=parameter_values)
t_eval = np.arange(0, 121, 1)
sim.solve(t_eval=t_eval)
sim.plot(["Current [A]", "Terminal voltage [V]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=120.0, step=1.2), Output()), _dom_classes=('…

In this notebook we have seen how we can change the parameters of our model. In [Tutorial 5](./Tutorial%205%20-%20Run%20experiments.ipynb) we show how can we define and run experiments.